Gaussian noise
We will use different filters, including median filters, mean filters, Gaussian filters, bilateral filters, inverse filters and Wiener filters to try to remove the noise. For each filter we will apply different window sizes and compare their results. For the result, we will use PSNR to measure the restoration quality. And we will discuss the result also based on the grayscale histogram and spectrum. 